In [17]:
import os
import warnings

from langchain.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain_openai import ChatOpenAI
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains.retrieval import create_retrieval_chain
from dotenv import load_dotenv

load_dotenv()
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY')

warnings.filterwarnings('ignore')

In [2]:
data = TextLoader('mmr_rag_dataset.txt')
raw_docs = data.load()
splitter = RecursiveCharacterTextSplitter(chunk_size=300, chunk_overlap=50)
chunks = splitter.split_documents(raw_docs)
embedding_model = OpenAIEmbeddings()
vector_store = FAISS.from_documents(chunks, embedding_model)

In [3]:
retriever = vector_store.as_retriever(search_type='mmr', search_kwargs={'k': 3})
retriever

VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x000002060E142850>, search_type='mmr', search_kwargs={'k': 3})

In [ ]:
prompt = PromptTemplate.from_template("""
Answer the question based on the context provided.
                                      
                                      Context:
                                      {context}

                                      Question:
                                      {input}

                                      
                                        """)

llm = ChatOpenAI(model='gpt-4o')
llm

ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x000002060E1492D0>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x000002060E149FD0>, root_client=<openai.OpenAI object at 0x000002060E148D50>, root_async_client=<openai.AsyncOpenAI object at 0x000002060E149D90>, model_name='gpt-4o', model_kwargs={}, openai_api_key=SecretStr('**********'))

In [16]:
document_chain = create_stuff_documents_chain(llm=llm, prompt=prompt)

In [15]:
rag_chain = create_retrieval_chain(
    retriever=retriever,
    combine_docs_chain=document_chain
)

In [7]:
query = {"input": "How does LangChain supports agent and memory?"}

response = rag_chain.invoke(query)
response

{'input': 'How does LangChain supports agent and memory?',
 'context': [Document(id='8cbefd99-d2e6-432c-94a4-9a365e915506', metadata={'source': 'mmr_rag_dataset.txt'}, page_content='Memory in LangChain helps models retain previous interactions, making multi-turn conversations more coherent.\nAgents in LangChain can use tools like calculators, search APIs, or custom functions based on the instructions they receive.'),
  Document(id='9cbc576a-6fa1-488a-9e39-ea2e8fd00ab0', metadata={'source': 'mmr_rag_dataset.txt'}, page_content='LangChain allows LLMs to act as agents that decide which tool to call and in what order during a task.\nLangChain supports conversational memory using ConversationBufferMemory and summarization memory with ConversationSummaryMemory.'),
  Document(id='254f18dc-df61-44cb-9256-7a278367e428', metadata={'source': 'mmr_rag_dataset.txt'}, page_content='BM25 and vector-based retrieval can be combined in LangChain to support hybrid retrieval strategies.\nFAISS is a high-p

In [8]:
print(response['answer'])

LangChain supports agents by allowing LLMs to act as decision-makers that determine which tools to use and in what sequence during a task. These agents can utilize tools such as calculators, search APIs, or custom functions based on their instructions.

When it comes to memory, LangChain enhances multi-turn conversation coherence with two types of memory: conversational memory and summarization memory. Conversational memory is managed using ConversationBufferMemory, while summarization memory is handled with ConversationSummaryMemory. This helps the models retain information from previous interactions, making future conversations more coherent and contextually aware.


In [14]:
for i, _ in enumerate(response):
    print(f"\nDoc: {i+1}")
    print(response['context'][i].page_content)
    


Doc: 1
Memory in LangChain helps models retain previous interactions, making multi-turn conversations more coherent.
Agents in LangChain can use tools like calculators, search APIs, or custom functions based on the instructions they receive.

Doc: 2
LangChain allows LLMs to act as agents that decide which tool to call and in what order during a task.
LangChain supports conversational memory using ConversationBufferMemory and summarization memory with ConversationSummaryMemory.

Doc: 3
BM25 and vector-based retrieval can be combined in LangChain to support hybrid retrieval strategies.
FAISS is a high-performance library for similarity search that LangChain leverages for efficient retrieval in RAG pipelines.
